## 機器學習百日馬拉松期中練習測驗

#### Overview

隨著移動設備的完善和普及，零售與電子商務進入了高速發展階段，這其中以 O2O（Online to Offline）消費最為熱絡。據統計，O2O 行業估值上億的創業公司至少有 10 家，也不乏百億巨頭的身影。O2O 行業每日自然流量有數億消費者，各類 APP 每天記錄了超過百億條用戶行為和位置記錄，因而成為大數據科研和商業化運營的最佳結合點之一。

以優惠券活化老用戶或吸引新客戶消費是 O2O 的一種重要營銷方式。然而，隨機投放的優惠券對多數用戶造成無意義的干擾。對商家而言，濫發的優惠券可能降低品牌聲譽，同時難以估算營銷成本。個性化投放是提高優惠券核銷率的重要技術，它可以讓具有一定偏好的消費者得到真正的實惠，同時賦予商家更強的營銷能力。本次練習數據擷取自電商之部分數據，希望各位通過分析建模，預測用戶是否會在規定時間內使用相應優惠券。

#### Data

本賽題提供用戶在2016年1月1日至2016年5月31日之間真實線下消費行為，預測用戶在2016年6月領取優惠券後15天以內的使用情況。

#### Evaluation

本賽題目標是預測投放的優惠券是否在規定時間內核銷。針對此任務及一些相關背景知識，以該用戶使用於某日取得之優惠券核銷預測 AUC（ROC 曲線下面積）作為評價標準。即對將 User_id - Date_received - Coupon_id 為一組計算核銷預測的AUC值，若某使用者於同一日取得多張相同優惠券，則任一張核銷皆為有效核銷。

#### Reference

[ml100marathon-02-01: Midterm exam for ML 100 marathon by Cupoy](https://www.kaggle.com/c/ml100marathon-02-01)

[ml100marathon-02-01: Baseline example](https://www.kaggle.com/ml100marathon/baseline-example)

## 準備工程

In [1]:
import os
import time
import functools
import numpy as np
import pandas as pd
from datetime import date

from IPython.display import display
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score

In [2]:
data_path = './data/part04/'

X_train = pd.read_csv(data_path + 'coupon_train.csv')
X_test = pd.read_csv(data_path + 'coupon_test.csv')

### 1. 消去資料集欄位 Coupon_id 或 Date_received 為空缺值的列，這些列無法組成題目要求的 uid。

In [3]:
X_train = X_train[~(X_train['Coupon_id'].isna() | X_train['Date_received'].isna())]
X_test = X_test[~(X_test['Coupon_id'].isna() | X_test['Date_received'].isna())]

### 2. 指定需要重複使用的參數。

In [4]:
X_all = pd.concat([X_train, X_test], axis=0, sort=False).reset_index(drop=True)

X_train_length = X_train.shape[0]

In [5]:
print(X_train.shape)
X_train.head()

(746969, 7)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN


In [6]:
print(X_test.shape)
X_test.head()

(306313, 6)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,1439408,4663,11002.0,150:20,1.0,20160528.0
1,1439408,2632,8591.0,20:1,0.0,20160613.0
3,1439408,2632,8591.0,20:1,0.0,20160516.0
4,2029232,450,1532.0,30:5,0.0,20160530.0
5,2029232,6459,12737.0,20:1,0.0,20160519.0


In [7]:
print(X_all.shape)
X_all.head()

(1053282, 7)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
1,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
2,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
3,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN
4,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN


In [8]:
X_all.dtypes

User_id            int64
Merchant_id        int64
Coupon_id        float64
Discount_rate     object
Distance         float64
Date_received    float64
Date             float64
dtype: object

## 特徵工程

### 1. 檢查空缺值

In [9]:
def check_nan(df):
    df_nan_ratio = (df.isnull().sum() / len(df)) * 100
    df_nan_ratio = df_nan_ratio.drop(df_nan_ratio[df_nan_ratio == 0].index).sort_values(ascending=False)
    df_nan_ratio = pd.DataFrame({'Missing Ratio': df_nan_ratio})
    display(df_nan_ratio.head(10))

check_nan(X_all)

,Missing Ratio
Date,95.923599
Distance,10.064066


### 2-1. 標記空缺值為 -1 於數值欄位

In [10]:
X_all['Date'] = X_all['Date'].replace({np.nan: -1}).astype('int64')

### 2-2. 使用中位數填補 Distance 欄位

In [11]:
X_all.loc[X_all['Distance'].isna(), ['Distance']] = X_all['Distance'].median()

### 2-3. 將明顯為類別資料的欄位轉成物件欄位

In [12]:
def cast_column_to_str(df, column):
    df[column] = df[column].astype('int64').astype(str)

cast_column_to_str(X_all, 'User_id')
cast_column_to_str(X_all, 'Merchant_id')
cast_column_to_str(X_all, 'Coupon_id')
cast_column_to_str(X_all, 'Date_received')
cast_column_to_str(X_all, 'Date')

### 2-4. 再次檢查空缺值

In [13]:
check_nan(X_all)

,Missing Ratio


In [14]:
X_all.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,8591,20:1,0.0,20160217,-1
1,1439408,2632,1078,20:1,0.0,20160319,-1
2,1832624,3381,7610,200:20,0.0,20160429,-1
3,2029232,3381,11951,200:20,1.0,20160129,-1
4,2223968,3381,9776,10:5,2.0,20160129,-1


In [15]:
X_all.dtypes

User_id           object
Merchant_id       object
Coupon_id         object
Discount_rate     object
Distance         float64
Date_received     object
Date              object
dtype: object

### 3-1. 新增欄位 uid

In [16]:
X_all['uid'] = X_all[['User_id', 'Coupon_id', 'Date_received']].apply(lambda x: '_'.join(x.values), axis=1)

### 3-2. 新增欄位 ui_dr_count、mi_dr_count、ci_dr_count

#### 分別以 User_id、Merchant_id、Coupon_id 分組並計數，這代表曾經擁有、發行、流通的優惠券數量。

In [17]:
def make_own_counts(df, new_column, by_column, target_column, default_value=0):
    df_temp = df.groupby([by_column], as_index=False).count()
    df_temp = df_temp.rename(columns={target_column: new_column})
    return pd.merge(df, df_temp[[by_column, new_column]], how='outer', sort=False, on=by_column)

X_all = make_own_counts(X_all, 'ui_dr_count', 'User_id', 'Date_received')
X_all = make_own_counts(X_all, 'mi_dr_count', 'Merchant_id', 'Date_received')
X_all = make_own_counts(X_all, 'ci_dr_count', 'Coupon_id', 'Date_received')

### 3-3. 新增欄位 ui_dt_count、mi_dt_count、ci_dt_count

#### 分別以 User_id、Merchant_id、Coupon_id 分組並計數非空值的 Date，這代表曾經使用或被使用的優惠券數量。

In [18]:
def make_usage_counts(df, new_column, by_column, target_column, select_condition, default_value=0):
    df_temp = df[select_condition].groupby([by_column], as_index=False).count()
    df_temp = df_temp.rename(columns={target_column: new_column})
    df_temp = pd.merge(df, df_temp[[by_column, new_column]], how='outer', sort=False, on=by_column)
    df_temp[new_column] = df_temp[new_column].replace({np.nan: default_value})
    return df_temp

select_condition = X_all['Date'] != '-1'
X_all = make_usage_counts(X_all, 'ui_dt_count', 'User_id', 'Date', select_condition)
X_all = make_usage_counts(X_all, 'mi_dt_count', 'Merchant_id', 'Date', select_condition)
X_all = make_usage_counts(X_all, 'ci_dt_count', 'Coupon_id', 'Date', select_condition)

### 3-4. 新增欄位 dr_type、dr_req_price、dr_off_price、dr_dsc_rate

#### 編碼折價率形式是浮點數者為 0，(\d+:\d+) 者為 1，觀察資料可知只有這兩種。

In [19]:
def get_dr_type(column):
    if ':' not in column:
        return 0
    return 1

X_all['dr_type'] = X_all['Discount_rate'].apply(get_dr_type)

#### 拆除 (\d+:\d+) 形式的折價率，指派到 dr_req_price、dr_off_price 兩個欄位上，分別代表滿 x 元折 y 元的優待。

In [20]:
def get_dr_req_price(row):
    return int(row['Discount_rate'].partition(':')[0])

def get_dr_off_price(row):
    return int(row['Discount_rate'].rpartition(':')[2])

select_condition = X_all['dr_type'] == 1
X_all['dr_req_price'] = X_all[select_condition].apply(get_dr_req_price, axis=1)
X_all['dr_off_price'] = X_all[select_condition].apply(get_dr_off_price, axis=1)

#### 填補 dr_req_price 的空缺值：眾數。

In [21]:
def get_no_nan_dr_req_price(df):
    return df[~df['dr_req_price'].isna()]['dr_req_price'].mode()[0]

X_all.loc[X_all['dr_req_price'].isna(), ['dr_req_price']] = get_no_nan_dr_req_price(X_all)

#### 填補 dr_off_price 的空缺值：dr_req_price * (1 - Discount_rate)。

In [22]:
def get_no_nan_dr_off_price(row):
    return row['dr_req_price'] * (1 - float(row['Discount_rate']))

select_condition = X_all['dr_type'] == 0
X_all.loc[X_all['dr_off_price'].isna(), ['dr_off_price']] = X_all[select_condition].apply(get_no_nan_dr_off_price, axis=1)

#### 新增 dr_dsc_rate 代表以浮點數表示的新折價率。

In [23]:
def get_dr_dsc_rate(row):
    return 1 - row['dr_off_price'] / row['dr_req_price']

X_all['dr_dsc_rate'] = X_all.apply(get_dr_dsc_rate, axis=1)

### 3-5. 新增欄位 dr_weekday、dr_weekday_type、dr_weekday_1、dr_weekday_2、...、dr_weekday_7

#### 將 Date_received 轉換成 dr_weekday 的欄位，意義為消費者是星期幾拿到該張折價券。

In [24]:
def get_dr_weekday(row):
    return pd.to_datetime(row['Date_received'], format='%Y%m%d').dayofweek + 1

X_all['dr_weekday'] = X_all.apply(get_dr_weekday, axis=1)

#### 將 dr_weekday 轉換成 dr_weekday_type 的欄位，意義為消費者拿到該張折價券是在平日或是假日。

In [25]:
weekend_day = set([6, 7])

def get_dr_weekday_type(row):
    return 1 if row['dr_weekday'] in weekend_day else 0

X_all['dr_weekday_type'] = X_all.apply(get_dr_weekday_type, axis=1)

#### 獨熱編碼 dr_weekday。

In [26]:
weekday_cols = ['dr_weekday_{}'.format(i) for i in range(1,8)]

def make_dr_weekday_n(df):
    df_temp = pd.get_dummies(df['dr_weekday'])
    df_temp.columns = weekday_cols
    return df_temp

X_all[weekday_cols] = make_dr_weekday_n(X_all)

### 4-1. 新增欄位 target

#### 訓練資料 Date 為 -1 或 Date - Date_received > 15 天者標記為 0，其餘標記為 1。

In [27]:
def get_target(row):
    if row['Date'] == '-1':
        return 0
    td = pd.to_datetime(row['Date'], format='%Y%m%d') - pd.to_datetime(row['Date_received'], format='%Y%m%d')
    if td > pd.Timedelta(15, 'D'):
        return 0
    return 1

X_all['target'] = X_all.apply(get_target, axis=1)

## 訓練模型

### 1. 資料保存與讀取

In [28]:
# X_all.to_pickle(data_path + 'coupon_well_done_dataframe.pkl')
X_all = pd.read_pickle(data_path + 'coupon_well_done_dataframe.pkl')
X_all.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,uid,ui_dr_count,mi_dr_count,...,dr_weekday,dr_weekday_type,dr_weekday_1,dr_weekday_2,dr_weekday_3,dr_weekday_4,dr_weekday_5,dr_weekday_6,dr_weekday_7,target
0,1439408,2632,8591,20:1,0.0,20160217,-1,1439408_8591_20160217,5,43,...,3,0,0,0,1,0,0,0,0,0
1,1439408,2632,8591,20:1,0.0,20160613,-1,1439408_8591_20160613,5,43,...,1,0,1,0,0,0,0,0,0,0
2,1439408,2632,8591,20:1,0.0,20160516,-1,1439408_8591_20160516,5,43,...,1,0,1,0,0,0,0,0,0,0
3,2082569,2632,8591,20:1,3.0,20160223,-1,2082569_8591_20160223,2,43,...,2,0,0,1,0,0,0,0,0,0
4,5162810,2632,8591,20:1,10.0,20160204,-1,5162810_8591_20160204,1,43,...,4,0,0,0,0,1,0,0,0,0


### 2. 永久去除特定欄位，分成訓練集、驗證集、識別碼、目標值

In [29]:
X_ids = X_all['uid']
y_all = X_all['target']
X_all = X_all.drop(columns=['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Date_received', 'Date', 'uid', 'target'])
X_all = MinMaxScaler().fit_transform(X_all)
X_all

array([[0.00000000e+00, 3.36134454e-02, 3.41927658e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.36134454e-02, 3.41927658e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.36134454e-02, 3.41927658e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.00000000e-01, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.00000000e-01, 8.40336134e-03, 8.14113471e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.00000000e-01, 8.40336134e-03, 8.14113471e-06, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [30]:
X_train, X_val, y_train, y_val = train_test_split(X_all[:X_train_length], y_all[:X_train_length],
                                                  test_size=0.15, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(634923, 20)
(634923,)
(112046, 20)
(112046,)


### 3. 建立模型

In [31]:
def print_time(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        print('Time elipsed:', int(time.time() - start_time), end='s\n')
        return result
    return wrapper

In [32]:
def print_score(estimator, X_train, y_train, X_val, y_val):
    cv_score = cross_val_score(estimator, X_train, y_train, cv=5).mean()
    auc_score = roc_auc_score(y_val, estimator.predict_proba(X_val)[:, 1])
    acc_score = accuracy_score(y_val, estimator.predict(X_val))
    print('CV score:', cv_score)
    print('AUC score:', auc_score)
    print('Acuuracy score:', acc_score)

In [33]:
@print_time
def make_estimator(X_train, y_train, X_val, y_val):
    gs_estimator = Pipeline(steps=[
        ('ss', StandardScaler()),
        ('en', SGDClassifier(loss='log', penalty='elasticnet', fit_intercept=True, max_iter=100,
                             shuffle=True, class_weight=None, n_jobs=-1, verbose=0),)
    ])
    gs_param_grid = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }
    gs_folder = StratifiedKFold(n_splits=3, shuffle=True)
    estimator = GridSearchCV(gs_estimator, gs_param_grid, cv=gs_folder, n_jobs=-1, verbose=0).fit(X_train, y_train)
    print_score(estimator, X_train, y_train, X_val, y_val)
    return estimator

estimator = make_estimator(X_train, y_train, X_val, y_val)

CV score: 0.9708389834871951
AUC score: 0.921607193526905
Acuuracy score: 0.9706459846848616
Time elipsed: 124s


In [34]:
@print_time
def make_simple_rf_estimator(X_train, y_train, X_val, y_val):
    estimator = RandomForestClassifier(n_estimators=100, max_depth=4, min_samples_split=2, min_samples_leaf=1,
                                       max_features=None, n_jobs=-1, bootstrap=True, verbose=0).fit(X_train, y_train)
    print_score(estimator, X_train, y_train, X_val, y_val)
    return estimator

estimator = make_simple_rf_estimator(X_train, y_train, X_val, y_val)

CV score: 0.9743212962673434
AUC score: 0.9813384871719955
Acuuracy score: 0.9744122949502882
Time elipsed: 181s


In [35]:
@print_time
def make_simple_gbdt_estimator(X_train, y_train, X_val, y_val):
    estimator = GradientBoostingClassifier(learning_rate=0.15, n_estimators=50, subsample=0.75,
                                           max_features=20, max_depth=6, tol=0.001, verbose=0).fit(X_train, y_train)
    print_score(estimator, X_train, y_train, X_val, y_val)
    return estimator

estimator = make_simple_gbdt_estimator(X_train, y_train, X_val, y_val)

CV score: 0.9821537416552075
AUC score: 0.9907068774002333
Acuuracy score: 0.9823376113382004
Time elipsed: 737s


## 輸出結果

#### 1. 測試資料似乎有部分在訓練資料裡，故以範本的 uid 為準進行合併。

#### 2. 測試和訓練資料皆有重複，造成篩選後仍有重複，故以 uid 分組並取 label 的平均為數據再合併作最後輸出。

In [36]:
def make_output(estimator, X_ids, X_all):
    X_test_ids = pd.read_csv(data_path + 'coupon_sample_submission.csv')[['uid']]
    y_test = pd.DataFrame(estimator.predict_proba(X_all)[:, 1], columns=['label'])
    output = pd.concat([X_ids, y_test], axis=1)
    output = pd.merge(output, X_test_ids, on=['uid'], how='inner')
    output['label'] = output.groupby('uid')['label'].transform('mean')
    output = output.drop_duplicates(['uid'])
    print(output.shape)
    display(output.head())
    return output

make_output(estimator, X_ids, X_all).to_csv('./output/Day_051-053_Mid_Exam_Coupon_HW_gbdt.csv', index=False)

(304096, 2)


,uid,label
0,1439408_8591_20160613,0.000028
1,1439408_8591_20160516,0.000028
2,5925919_8591_20160514,0.000028
3,7207738_8591_20160602,0.000028
4,2423386_8591_20160531,0.000028
